Let's start by creating a [Knowledge Base for Amazon Bedrock](https://aws.amazon.com/bedrock/knowledge-bases/) 
to provide knowledge about mortgages. In this notebook, we will create a knowledge base using the content available in the mortgage_dataset folder. This knowledge base will be used by Mortgage assistant agent to answer general QnA. 

Step 1: Install and import the libraries required

In [ ]:
# Dependencies are managed by uv via pyproject.toml
# Run 'uv sync' in terminal to install all dependencies
# Update the Kernel to pint to the new uv that's created as part of prerequisite
print("Dependencies installed via uv sync")

In [1]:
import os
import time
import boto3
import logging
import botocore
import json
from textwrap import dedent

%load_ext autoreload
%autoreload 2

In the following cell, we add and `knowledge_base_helper` on Python path. This provides functionality for creating the knowledge base if it does not already exists.



In [2]:
import sys
sys.path.insert(0, '..') 


from src.utils.knowledge_base_helper import KnowledgeBasesForAmazonBedrock

kb = KnowledgeBasesForAmazonBedrock()

Create boto3 clients

In [3]:
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

Get the region and bucket name. The bucket will be created if its not present already.

In [4]:
region = boto3.session.Session().region_name
account_id = sts_client.get_caller_identity()["Account"]
suffix = f"{region}-{account_id}"
bucket_name = f'agentcore-workshop-{suffix}'

In [ ]:
agent_foundation_model = ["us.anthropic.claude-3-7-sonnet-20250219-v1:0"]

### Create Knowledge Base 
 We will now create the knowledge base with Amazon OpenSearch Serverless as the vector store. To do so, we will use the helper class `KnowledgeBasesForAmazonBedrock` which creates the knowledge base and all of its prerequisites:
1. IAM roles and policies
2. S3 bucket
3. Amazon OpenSearch Serverless encryption, network and data access policies
4. Amazon OpenSearch Serverless collection
5. Amazon OpenSearch Serverless vector index
6. Knowledge Base
7. Knowledge Base data source

This might take a few minutes, so have a break!

In [8]:
knowledge_base_name = "mortgage-agent-kb-test1"

knowledge_base_description = "KB containing information on mortgages"

In [9]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")


Creating KB mortgage-agent-kb-test1
Step 1 - Creating or retrieving agentcore-workshop-us-west-2-266735823905 S3 bucket for Knowledge Base documents
Bucket agentcore-workshop-us-west-2-266735823905 already exists - retrieving it!
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_303) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '319',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Wed, 03 Sep 2025 04:55:24 '
                                                 'GMT',
                                         'x-amzn-requestid': '6648eeb8-657d-4514-9de6-618202cabc7e'},
                        'HTTPStatusCode': 200,
     

In [ ]:
# function to upload to S3 bucket
import boto3

def upload_file_to_s3(file_path, bucket_name, object_key=None):
    """Upload a file to S3 bucket"""
    s3_client = boto3.client('s3')
    
    # Check if bucket exists, create if not
    existing_buckets = [bucket['Name'] for bucket in s3_client.list_buckets()['Buckets']]
    if bucket_name not in existing_buckets:
        s3_client.create_bucket(Bucket=bucket_name)
    
    if object_key is None:
        object_key = file_path.split('/')[-1]
    
    s3_client.upload_file(file_path, bucket_name, object_key)
    return f"s3://{bucket_name}/{object_key}"

In [ ]:
upload_file_to_s3("mortgage_dataset/15-Year vs. 30-Year Mortgage What's the Difference .html", bucket_name,"15-Year vs. 30-Year Mortgage What's the Difference .html") 

In [ ]:
upload_file_to_s3("mortgage_dataset/Mortgage Refinancing When Does It Make Sense .html", bucket_name,"Mortgage Refinancing When Does It Make Sense .html")

Now we ingest the documents, which chunks the source documents and stores an embedding for each chunk into the underying knowledge base vector store. For a simple example, this ingestion takes a couple minutes.

In [10]:
%%time
# Start an ingestion job to synchronize data
kb.synchronize_data(kb_id, ds_id)
print('KB synchronization completed\n')

{ 'dataSourceId': 'W0DC94D6DG',
  'ingestionJobId': 'WUNLGASBWN',
  'knowledgeBaseId': 'QVBZHOLKNO',
  'startedAt': datetime.datetime(2025, 9, 3, 5, 1, 36, 992240, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 9, 3, 5, 1, 36, 992240, tzinfo=tzutc())}
{ 'dataSourceId': 'W0DC94D6DG',
  'ingestionJobId': 'WUNLGASBWN',
  'knowledgeBaseId': 'QVBZHOLKNO',
  'startedAt': datetime.datetime(2025, 9, 3, 5, 1, 36, 992240, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 2,
                 

### Test the Knowledge Base
Now the Knowledge Base is available we can test it out using the [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) functions. 

#### Testing Knowledge Base with Retrieve and Generate API

Let's first test the knowledge base using the retrieve and generate API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a Bedrock LLM.

In [11]:
f"arn:aws:bedrock:{region}:{account_id}:inference-profile/{agent_foundation_model[0]}"

'arn:aws:bedrock:us-west-2:266735823905:inference-profile/us.anthropic.claude-3-5-haiku-20241022-v1:0'

#### Please be aware, the sync operation may take a few minutes to complete. Before the first sync finishes, you will not get an answer when you call `retrieve_and_generate` or `retrieve`. If that is the case, wait a few minutes and try again.

In [12]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "compare and contrast 15-year vs 30-year mortgage type"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": f"arn:aws:bedrock:{region}:{account_id}:inference-profile/{agent_foundation_model[0]}",
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

A 15-year and 30-year mortgage differ primarily in their loan term and monthly payment structure. The 15-year mortgage has higher monthly payments but costs significantly less in total interest over the life of the loan. In contrast, a 30-year mortgage offers lower monthly payments, making it more affordable for many homebuyers, but results in paying almost double the total interest compared to a 15-year mortgage. With a 15-year mortgage, borrowers typically pay a lower interest rate, sometimes up to a full percentage point less than a 30-year loan. The shorter term means that more of each payment goes toward the principal earlier, helping homeowners build equity faster. Banks also consider 15-year loans less risky, which contributes to the lower interest rates. The primary advantage of a 30-year mortgage is flexibility. Lower monthly payments allow borrowers to potentially buy a more expensive home, build savings, or allocate funds to other financial goals. However, this comes at the 

As you can see, with the retrieve and generate API we get the final response directly and we don't see the different sources used to generate this response. Let's now retrieve the source information from the knowledge base with the retrieve API.

**Testing Knowledge Base with Retrieve API**

If you need an extra layer of control, you can retrieve the chuncks that best match your query using the retrieve API. In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata.

In [ ]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":3,
        } 
    },
    retrievalQuery={
        'text': 'What are the cons of a 15-year mortgage?'
    }
)

def response_print(retrieve_resp):
    #structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print('-----------------------------------------------------------------------------------------')
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

Store knowledge base ID and name for subsequent labs

In [13]:
kb_id

'QVBZHOLKNO'

Store the knowledge base id **kb_id** in AWS Parameter Store. It will be accessed in the labs that you will be running.

In [14]:

param_name = '/app/mortgage_assistant/agentcore/kb_id'

ssm = boto3.client("ssm")
ssm.put_parameter(Name=param_name, Value=kb_id, Type="String", Overwrite=True)
print(f"Stored {kb_id} in SSM: {param_name}")

Stored QVBZHOLKNO in SSM: /app/mortgage_assistant/agentcore/kb_id


In this lab, we created a knowledge base that will be used by one of the agents to answers queries on mortgage 